In [1]:
%load_ext lab_black

from puzzles import load

import numpy as np

In [54]:
class Node:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.parents = []
        self.visited = False

    def all_parents_visited(self) -> bool:
        if len(self.parents) == 0:
            return True
        return all([p.visited for p in self.parents])

    def __repr__(self):
        child_name = ", ".join([x.name for x in self.children])
        par_name = ", ".join([x.name for x in self.parents])
        return f"Node('{self.name}', parents={par_name}, next={child_name})"

    def __lt__(self, other):
        return self.name < other.name

    def __gt__(self, other):
        return self.name > other.name

    def __le__(self, other):
        return self.name <= other.name

    def __ge__(self, other):
        return self.name >= other.name

In [64]:
# data = """Step C must be finished before step A can begin.
# Step C must be finished before step F can begin.
# Step A must be finished before step B can begin.
# Step A must be finished before step D can begin.
# Step B must be finished before step E can begin.
# Step D must be finished before step E can begin.
# Step F must be finished before step E can begin."""

data = load(7)

nodes = {}

for line in data.strip().split("\n"):

    tokens = line.split()
    a, b = tokens[1], tokens[-3]

    nodeA = Node(a) if a not in nodes else nodes[a]
    nodeB = Node(b) if b not in nodes else nodes[b]

    nodeA.children.append(nodeB)
    nodeB.parents.append(nodeA)

    nodes[a] = nodeA
    nodes[b] = nodeB

nodes

{'V': Node('V', parents=, next=H, O, A, D, Q, W, M, Z),
 'H': Node('H', parents=V, J, O, R, next=I, Q, Y, S),
 'U': Node('U', parents=, next=R, N, Z, G, F, O, A),
 'R': Node('R', parents=U, B, T, W, next=S, J, I, H, C, O),
 'E': Node('E', parents=, next=D, J, S),
 'D': Node('D', parents=E, V, B, A, next=C, Z, S, O),
 'B': Node('B', parents=, next=R, T, K, D, S),
 'W': Node('W', parents=V, next=X, P, R, M, G, K),
 'X': Node('X', parents=W, G, J, next=Q, S, K),
 'A': Node('A', parents=V, U, next=P, C, D, Z, I),
 'P': Node('P', parents=A, W, T, next=Y, S, J, L, Z),
 'T': Node('T', parents=B, next=L, S, R, P, F),
 'L': Node('L', parents=T, P, next=J, I),
 'F': Node('F', parents=U, T, next=C, S, K, J, O),
 'C': Node('C', parents=F, D, A, Q, G, R, Z, M, next=Y),
 'Y': Node('Y', parents=P, K, I, S, C, G, Q, H, next=),
 'N': Node('N', parents=U, next=G, Z),
 'G': Node('G', parents=N, U, W, next=K, I, Y, X, C),
 'S': Node('S', parents=R, Q, T, P, I, F, Z, X, B, D, K, E, H, J, next=Y),
 'O': Nod

In [66]:
def reset_nodes(nodes):
    for n in nodes.values():
        n.visited = False


reset_nodes(nodes)
result_string = ""

head_nodes = sorted([v for v in nodes.values() if len(v.parents) == 0])

while len(head_nodes) > 0:
    processing = head_nodes.pop(0)
    processing.visited = True
    result_string += processing.name

    for child in processing.children:
        if child.all_parents_visited():
            head_nodes.append(child)

    head_nodes = sorted(head_nodes)

result_string

'BETUFNVADWGPLRJOHMXKZQCISY'

---

In [2]:
class Node:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.parents = []
        self.timer = ord(self.name) - ord("A") + 1 + 60
        self.time_left = self.timer

    def has_parents(self):
        return len(self.parents) > 0

    def __repr__(self):
        p = (
            "None"
            if len(self.parents) == 0
            else ",".join([c.name for c in self.parents])
        )
        c = (
            "None"
            if len(self.children) == 0
            else ",".join([c.name for c in self.children])
        )
        return f"Node('{self.name}' [{self.time_left}/{self.timer}],  parent={p}, children={c})"

    def __eq__(self, other):
        return self.name == other.name

    def __lt__(self, other):  # less than
        return self.name < other.name

In [3]:
all_nodes = {}

for line in load(7).strip().split("\n"):

    tokens = line.split()
    parent_name, child_name = tokens[1], tokens[-3]

    if parent_name not in all_nodes:
        parent_node = Node(parent_name)
        all_nodes[parent_name] = parent_node
    else:
        parent_node = all_nodes[parent_name]

    if child_name not in all_nodes:
        child_node = Node(child_name)
        all_nodes[child_name] = child_node
    else:
        child_node = all_nodes[child_name]

    if parent_node not in child_node.parents:
        child_node.parents.append(parent_node)

    if child_node not in parent_node.children:
        parent_node.children.append(child_node)

In [4]:
NUM_WORKERS = 5

# список "рабочих" = обрабатываемые сейчас узлы
workers = []

# сколько всего времени потрачено
T = 0

# список обработанных узлов
processed = set()

# начальная очередь — список узлов без родителей, сортированный по алфавиту
queue = []
for node in all_nodes.values():
    if not node.has_parents():
        queue.append(node)
queue = sorted(queue)


while True:
    # рассмотрим два варианта: 5 рабочих и 2 таска или наоборот,
    # 2 свободных рабочих и 5 тасков, итог один — 2 таска будут
    # разданы 2м рабочим, поэтому и min(.., ..)
    num_workers_free = NUM_WORKERS - len(workers)
    for _ in range(min(num_workers_free, len(queue))):
        workers.append(queue.pop(0))

    # сколько времени надо ждать до ближайшего события
    tm = min([x.time_left for x in workers])

    T += tm
    names_done = []
    i = 0

    # обновляем все узлы: вычитаем у них это время, убираем
    # из обрабатываемых, если сделались
    while i < len(workers):
        node = workers[i]
        node.time_left -= tm

        if node.time_left == 0:

            # запоминаем имена обработанных
            processed.add(node.name)
            names_done.append(node.name)
            del workers[i]

            # добавляем новые в очередь, если всех родителей обработали
            queue.extend(
                [
                    c
                    for c in node.children
                    if all([p.name in processed for p in c.parents])
                ]
            )
            queue = sorted(queue)
        else:
            i += 1

    print("".join(sorted(names_done)), end="")

    # условие выхода из цикла — все закончилось, и запас, и работа
    if len(queue) == 0 and len(workers) == 0:
        break

T

BEUVTANWDFGPRLOJMHXZKQCISY

848